<a href="https://colab.research.google.com/github/Chingkai-Lin/python-notes-machinelearning/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

先到kaggle的compete下載titanic的資料

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0930/raw/master/titanic/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0930/raw/master/titanic/test.csv"
urlretrieve(url, "test.csv")

('test.csv', <http.client.HTTPMessage at 0x7f639f2e0588>)

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

1. 補缺失值 \\
  pandas.concat可以合併兩資料

In [ ]:
datas = pd.concat([train_df, test_df], axis=0, ignore_index=True)

丟掉「乘客id」以及「存活」的資料

In [ ]:
datas = datas.drop(["PassengerId", "Survived"], axis=1)

數值：Age、SibSP、Parch、Tichet(數出有多少人有同一張票) 、Fare(票價) \\
類別型態：Pclass艙等、Name(mid)、Sex、Cabin、Embarked(出發地)

In [ ]:
# .isna() 檢查資料是不是空的
na = datas.isna().sum()
# 篩選大0的 Series[True/False List/series]  , sort_values排序
na[na > 0].sort_values(ascending=False)

Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

先補Embarked \\
找出該欄的資料，補上最大值的

In [ ]:
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [ ]:
# 先處理Ticket
dic = datas["Ticket"].value_counts()
def count(n):
    return dic[n]
datas["Ticket"] = datas["Ticket"].apply(count)

In [ ]:
datas["Ticket"]

0       1
1       2
2       1
3       2
4       1
       ..
1304    1
1305    3
1306    1
1307    1
1308    3
Name: Ticket, Length: 1309, dtype: int64

補中位數 \\
類別補出現最多的，數值補中位數 \\
Pclass需要補最多出現的，不能是中位數

In [ ]:
med = datas.median().drop(["Pclass"])
datas = datas.fillna(med)

In [ ]:
# 一個段落，看一下還缺多少
na = datas.isna().sum()
# 篩選 Series[True/False list/series]
na[na > 0].sort_values(ascending=False)

Cabin    1014
dtype: int64

In [ ]:
# 處理Cabin
def cabin(c):
  if pd.isna(c):
    return None
  else:
    return c[0]
datas["Cabin"] = datas["Cabin"].apply(cabin)

類別轉數字，One-Hot Encoding \\
Cabin三個 \\
Pclass: 可以不做One-Hot Encoding，因為他的三個艙等有大小關係

In [ ]:
def name(n):
    mid = n.split(",")[-1].split(".")[0]
    mid = mid.strip()
    return mid
middle = datas["Name"].apply(name)
counts = middle.value_counts()
reserved = counts[:4].index
def name2(mid):
    if mid in reserved:
        return mid
    else:
        return None
datas["Name"] = middle.apply(name2)

In [ ]:
datas = pd.get_dummies(datas)
datas = pd.get_dummies(datas, columns=["Pclass"])

In [ ]:
datas["Family"] = datas["SibSp"] + datas["Parch"]

In [ ]:
datas

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [ ]:
# 將data切回來，
# [first row, second row......]
x_train = datas.iloc[:len(train_df)]
x_predict = datas.iloc[len(train_df):]
y_train = train_df["Survived"]
x_predict

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
891,34.5,0,0,1,7.8292,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
892,47.0,1,0,1,7.0000,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
893,62.0,0,0,1,9.6875,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
894,27.0,0,0,1,8.6625,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
895,22.0,1,1,2,12.2875,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


## 隨機森林 RandomForestClassifier
避免過度擬和 \\
常調整參數n_estimators(幾顆決策樹)、max_depth(決策樹分幾層) \\
## 交叉驗證cross validation：（多做幾次）
  1. 假設共1000筆資料，分成10份 \\
  2-1. 0\~8區訓練，9驗證 \\
  2-2. 0~7,9區訓練，8驗證 \\
  ... \\
  取平均

In [ ]:
# GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
    "n_estimators":range(20, 100),
    "max_depth":range(4, 11)
}
clf = RandomForestClassifier(random_state=1)
cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train, y_train)
print(cv.best_score_)
print(cv.best_params_)


0.8417602996254681
{'max_depth': 8, 'n_estimators': 63}


In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=63, max_depth=8, random_state=1)
# cv=10 分10段做10次 n_jobs=-1 用最多核心同時處理
scores = cross_val_score(clf, x_train, y_train, cv=10, n_jobs=-1)
print(scores)
print(np.average(scores))

[0.83333333 0.8988764  0.73033708 0.91011236 0.88764045 0.83146067
 0.85393258 0.78651685 0.87640449 0.80898876]
0.8417602996254681


In [ ]:
clf = RandomForestClassifier(n_estimators=63, max_depth=7)
clf.fit(x_train, y_train)
pre = clf.predict(x_predict)
df = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
df.to_csv("rf.csv", encoding="utf-8", index=False)

## Scaling 將所有數值都變為0-1的數值
避免像年齡金額等等巨大的差異影響到結果//
嘗試不用決策樹，用不同演算法




In [ ]:
# 用scaling將所有數值變為0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
datas_scale = scaler.fit_transform(datas)
datas_scale = pd.DataFrame(datas_scale, columns=datas.columns)
x_train_scale = datas_scale.iloc[:len(train_df)]
x_predict_scale = datas_scale.iloc[len(train_df):]
x_predict_scale

In [ ]:
# GridSearchCV
# nearest neighbors找鄰近的資料 沒有random
# kneighbors 要做normalized
from sklearn.neighbors import KNeighborsClassifier
params = {
    "n_neighbors":range(5, 100),
}
clf = KNeighborsClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train_scale, y_train)
print(cv.best_score_)
print(cv.best_params_)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=11)
clf.fit(x_train_scale, y_train)
pre = clf.predict(x_predict_scale)
df = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
df.to_csv("knn.csv", encoding="utf-8", index=False)